# Chap09 - 텐서플로 시작하기

## SetUp

In [1]:
import os
import numpy as np
import tensorflow as tf

# 일관된 출력을 위해 유사난수 초기화
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sn
sn.set()
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# 한글출력
# matplotlib.rc('font', family='AppleGothic')  # MacOS
matplotlib.rc('font', family='Malgun Gothic')  # Windows
plt.rcParams['axes.unicode_minus'] = False

## 9.1 Install

```bash
# TensorFlow CPU 
$ pip install tensorflow

# TensorFlow GPU
$ pip install tensorflow-gpu
```

In [2]:
import tensorflow as tf

tf.__version__

'1.10.0'

## 9.2 첫 번째 계산 그래프를 만들어 세션에서 실행하기

In [4]:
import tensorflow as tf

reset_graph()

x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

In [5]:
f

<tf.Tensor 'add_1:0' shape=() dtype=int32>

In [6]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


In [7]:
# Session 닫기
sess.close()

위의 방법을 `with` 스코프를 이용해 간단하게 작성할 수 있다. `with` 스코프가 끝나면 `tf.Session`은 자동 종료 된다.

In [8]:
with tf.Session() as sess:
    x.initializer.run()  # tf.get_default_session().run(x.initializer)
    y.initializer.run()
    result = f.eval()  # tf.get_default_session().run(f)
    
print(result)

42


각 변수를 일일이 초기화하는 대신 `global_variables_initializer()`함수를 사용할 수 있다.

In [13]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    # 모든 변수 초기화
    init.run()  # sess.run(init)
    result = f.eval()

print(result)

42


## 9.3 계산 그래프 관리

노드를 만들면 자동으로 기본 계산 그래프에 추가된다.

In [15]:
reset_graph()

x1 = tf.constant(1)
x1.graph is tf.get_default_graph()

True

`tf.Graph()`를 이용해 새로운 독립적인 계산 그래프를 만들 수 있으며, 이를 `.as_default()`를 통해 기본 계산 그래프로 사용할 수 있다.

In [20]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.constant(2)
    print(x2.graph is tf.get_default_graph())  # True
    
print(x2.graph is tf.get_default_graph())  # False
print(x2.graph is graph)  # True

True
False
True


## 9.4 노드 값의 생애주기

> 노드는 연산, 엣지는 텐서 객체!!

텐서플로는 그래프의 연결 상태를 기반으로 연산을 최적화한다. 각 그래프에는 노드 간에 의존관계(dependency)가 존재한다. 예를 들어, 아래의 그림 'A'에서 노드 `e`는 노드 `c`에 직접의존(direct dependeny)하고 있고, 노드 `a`에는 간접의존(indirect dependency) 한다.

![](./images/graph02.png)

위의 그림에서 노드`e`를 계산하기 위해서는 노드 `c, b, a`만 계산 해주면 된다. 따라서, 의존관계를 이용해 연산량이 최소화할 수 있다. 이처럼 그래프를 통해 각 노드의 모든 의존관계를 파악할 수 있다.

In [22]:
w = tf.constant(3)
x = w + 2
y = tf.add(x, 5)
z = x * 3

with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)  # 10
    print(z_val)  # 15

10
15


## 9.5 텐서플로를 이용한 선형 회귀

### 정규식을 이용한 선형 회귀 풀이

$$
\hat{\mathbf{W}} = \left(\mathbf{X}^{T} \mathbf{X} \right)^{-1} \mathbf{X}^{T} \mathbf{Y} 
$$

In [30]:
import numpy as np
from sklearn.datasets import fetch_california_housing

reset_graph()

housing = fetch_california_housing()
m, n = housing.data.shape
# bias 추가
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

print('data shape :', housing.data.shape)
print('data_plus_bias shape :', housing_data_plus_bias.shape)

data shape : (20640, 8)
data_plus_bias shape : (20640, 9)


In [33]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')  # (20640, 1)
X_t = tf.transpose(X)
weight = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(X_t, X)), X_t), y)

with tf.Session() as sess:
    weight_val = weight.eval()
    
print(weight_val)

[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]


### Scikit-Learn과 비교

In [34]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing.data, housing.target.reshape(-1, 1))

print(np.r_[lin_reg.intercept_.reshape(-1, 1), lin_reg.coef_.T])

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]
